In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests
import time
import subprocess
import re
from google.cloud import bigquery
import datetime
from pyspark.sql.functions import lit, unix_timestamp


In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords=nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# parameter
IP = 'localhost'    # ip port
PORT = 9001       # port

STREAMTIME = 6000
 
timemark=datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

In [4]:
def update_state(value,historyvalue):
    if historyvalue is None:
        historyvalue=0
    return sum(value,historyvalue)

In [5]:
def wordCount(words):
    counting=words.map(lambda w:w.lower()).filter(lambda w:w not in stopwords).map(lambda w:(w,1)) \
    .updateStateByKey(update_state).transform(lambda wordmap:wordmap.sortBy(lambda word:word[1],ascending=False))
    return counting                                                                        

In [6]:
if __name__ == '__main__':
    # Initialization
    conf = SparkConf()
    conf.setMaster('local[2]')
    conf.setAppName("TwitterStreamApp")

    sc = SparkContext(conf=conf)
    sc.setLogLevel("ERROR")

    sql_context = SQLContext(sc)

    ssc = StreamingContext(sc, 5)
    ssc.checkpoint("~/checkpoint_TwitterApp")

    # listen data
    dataStream = ssc.socketTextStream(IP, PORT)
    dataStream.pprint()

    words = dataStream.flatMap(lambda line: line.split(" "))

    # word count
    wordCount = wordCount(words)
    wordCount.pprint()

    # start
    ssc.start()
    time.sleep(STREAMTIME)
    ssc.stop(stopSparkContext=False, stopGraceFully=True)


-------------------------------------------
Time: 2022-05-04 14:29:40
-------------------------------------------
RT @CSimmsQB: How did this happen??? How did the @Chiefs get Skyy Moore. Holy crap Batman. This guys is a baller best pure slot in the draf…RT @reblmoon: WE ALL CAN AGREE THAT THE MOONKNIGHT EPISODE 6 THESE SHOTS WERE CHEF'S KISS #Moonknight https://t.co/rLoGqhF196RT @RT3NENBAUM: #MoonKnight spoilers
•
•
•
•
hottest marvel couple https://t.co/DKXx12wXb7#JusticeLeagueFacts Pese a haber salvado a Superman, el asesinato de Maxwell Lord por parte de Wonder Woman género… https://t.co/kaK6WDqMAgRT @prettychicken13: ตามนี้5555555 #MoonKnight https://t.co/iXcd58mBntRT @elisstonee: #MoonKnight Perfeição https://t.co/ApHbuz1pwR@blurayangel Batman the animated seriesEmang bener Oscar Isaac gabakal balik maen Marc Spector di #MoonKnight
Soalnya dia udah jadi #JakeLockley 😂😂😂
Wanja… https://t.co/DUHEClkiW7RT @heonxkyun: i'm 
○ single
○ taken
...

----------------------------------------

KeyboardInterrupt: 